In [9]:
pip install contractions

     |████████████████████████████████| 321 kB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 283 kB 6.0 MB/s eta 0:00:01
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-macosx_10_9_x86_64.whl size=32256 sha256=c13c6a033daa1006846005a42652dfdd5f30fc8183c5e382392c9af019826080
  Stored in directory: /Users/manjusri/Library/Caches/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick
Note: you may need to restart the kernel to use updated packages.


In [40]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from nltk import ToktokTokenizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from bs4 import BeautifulSoup
import unicodedata
import re
import contractions
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to /Users/manjusri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/manjusri/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/manjusri/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
reviews_data = pd.read_csv('labeledTrainData.tsv', header=0, delimiter='\t')
pd.set_option('display.max_colwidth', None)
reviews_data.head()

,id,sentiment,review
0,5814_8,1,"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally starts is only on for 20 minutes or so excluding the Smooth Criminal sequence and Joe Pesci is convincing as a psychopathic all powerful drug lord. Why he wants MJ dead so bad is beyond me. Because MJ overheard his plans? Nah, Joe Pesci's character ranted that he wanted people to know it is he who is supplying drugs etc so i dunno, maybe he just hates MJ's music.<br /><br />Lots of cool things in this like MJ turning into a car and a robot and the whole Speed Demon sequence. Also, the director must have had the patience of a saint when it came to filming the kiddy Bad sequence as usually directors hate working with one kid let alone a whole bunch of them performing a complex dance scene.<br /><br />Bottom line, this movie is for people who like MJ on one level or another (which i think is most people). If not, then stay away. It does try and give off a wholesome message and ironically MJ's bestest buddy in this movie is a girl! Michael Jackson is truly one of the most talented people ever to grace this planet but is he guilty? Well, with all the attention i've gave this subject....hmmm well i don't know because people can be different behind closed doors, i know this for a fact. He is either an extremely nice but stupid guy or one of the most sickest liars. I hope he is not the latter."
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hines is a very entertaining film that obviously goes to great effort and lengths to faithfully recreate H. G. Wells' classic book. Mr. Hines succeeds in doing so. I, and those who watched his film with me, appreciated the fact that it was not the standard, predictable Hollywood fare that comes out every year, e.g. the Spielberg version with Tom Cruise that had only the slightest resemblance to the book. Obviously, everyone looks for different things in a movie. Those who envision themselves as amateur \""critics\"" look only to criticize everything they can. Others rate a movie on more important bases,like being entertained, which is why most people never agree with the \""critics\"". We enjoyed the effort Mr. Hines put into being faithful to H.G. Wells' classic novel, and we found it to be very entertaining. This made it easy to overlook what the \""critics\"" perceive to be its shortcomings."""
2,7759_3,0,"The film starts with a manager (Nicholas Bell) giving welcome investors (Robert Carradine) to Primal Park . A secret project mutating a primal animal using fossilized DNA, like ¨Jurassik Park¨, and some scientists resurrect one of nature's most fearsome predators, the Sabretooth tiger or Smilodon . Scientific ambition turns deadly, however, and when the high voltage fence is opened the creature escape and begins savagely stalking its prey - the human visitors , tourists and scientific.Meanwhile some youngsters enter in the restricted area of the security center and are attacked by a pack of large pre-historical animals which are deadlier and bigger . In addition , a security

In [ ]:
reviews_data

In [25]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

def transform_accented_chars(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

def tokenize(text):
    toktok = ToktokTokenizer()
    return toktok.tokenize(text)

def remove_stop_words(text):
    tokens = tokenize(text)
    stop_words = nltk.corpus.stopwords.words('english')
    return " ".join([token for token in tokens if token not in stop_words])

def remove_special_chars(text):
    return re.sub(r'[^a-zA-z]', ' ', text)

def remove_contractions(text):
    return contractions.fix(text)

def to_lower(text):
    tokens = tokenize(text)
    return " ".join([token.lower() for token in tokens])

def remove_punctuations(text):
    tokens = tokenize(text)
    return " ".join([re.sub(r'[^\w\s]','', token) for token in tokens])

def perform_lemmatization(text):
    tokens = tokenize(text)
    lemmatizer = nltk.stem.WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(token) for token in tokens])

def preprocess(df):
    reviews = df['review']
    preprocessed_text = []
    for review in reviews:
        review = remove_html_tags(review)
        review = to_lower(review)
        review = transform_accented_chars(review)
        review = remove_special_chars(review)
        review = remove_contractions(review)
        review = remove_punctuations(review)
        review = remove_stop_words(review)
        review = perform_lemmatization(review)
        
        preprocessed_text.append(review)
    return preprocessed_text

In [26]:
preprocessed_text = preprocess(reviews_data)
preprocessed_text[0]

'stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighty maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle message mj feeling towards press also obvious message drug bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fan would say made fan true really nice actual feature film bit finally start minute excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord want mj dead bad beyond mj overheard plan nah joe pesci character ranted wanted people know supplying drug etc dunno maybe hate mj music lot cool thing like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually director hate working one kid let a

In [31]:
def vectorize(data):
    vectorizer = TfidfVectorizer(max_features=1500)
    train_data_features = vectorizer.fit_transform(data)
    return train_data_features.toarray()
vectorized_data = vectorize(preprocessed_text)

In [32]:
vectorized_data.shape

(25000, 1500)

In [33]:
vectorized_data[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [35]:
X = vectorized_data
y = reviews_data['sentiment']

In [37]:
X.shape, y.shape

((25000, 1500), (25000,))

In [38]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=7)

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(17500, 1500)
(17500,)
(7500, 1500)
(7500,)


In [45]:
def construct_models_list(xtrain, xval, ytrain, yval):
    models = [
        {'model': LogisticRegression(solver='liblinear', random_state=7), 'name': 'Logistic Regressor'},
        {'model': SVC(kernel='linear', random_state=7), 'name': 'Logistic Regressor'},
        {'model': DecisionTreeClassifier(random_state=7), 'name': 'DecisionTree Classifier'},
        {'model': RandomForestClassifier(n_estimators=100, 
                                         max_depth=4, 
                                         n_jobs=4, 
                                         random_state=7), 'name': 'RandomForest Classifier'},
        {'model': AdaBoostClassifier(base_estimator=DecisionTreeClassifier(random_state=7), 
                                     n_estimators=100,
                                     random_state=7), 'name': 'AdaBoost Classifier'},
        {'model': GradientBoostingClassifier(n_estimators=100,
                                             max_depth=4,
                                             random_state=7), 'name': 'GradientBoost Classifier'},
        {'model': BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=7),
                                    n_jobs=4,
                                    random_state=7
                                   ), 'name': 'Bagging Classifier'}
    ]
    
    return models

In [46]:
def evaluate_models(xtrain, xval, ytrain, yval):
    result_df = pd.DataFrame(columns=['train accuracy', 'k-fold-cv mean', 'k-fold-cv std'])
    kfold = KFold(n_splits=10, random_state=7, shuffle=True)
    models = construct_models_list(xtrain, xval, ytrain, yval)
    for model_obj in models:
        model = model_obj['model']
        name = model_obj['name']
        model.fit(xtrain, ytrain)
        train_accuracy = model.score(xtrain, ytrain)
        cv_result = cross_val_score(model, xval, yval, cv=kfold)
        print(cv_result)
        result_df.loc[name] = [train_accuracy, cv_result.mean(), cv_result.std()]
    return result_df

In [47]:
result_df = evaluate_models(X_train, X_val, y_train, y_val)
result_df

[0.87066667 0.84266667 0.83733333 0.868      0.85466667 0.84266667
 0.848      0.85333333 0.85466667 0.85466667]
[0.852      0.84       0.83333333 0.86666667 0.85466667 0.832
 0.848      0.844      0.848      0.84133333]
[0.68133333 0.70133333 0.69066667 0.71733333 0.68       0.69866667
 0.67066667 0.688      0.71733333 0.69466667]
[0.816      0.78666667 0.796      0.804      0.77066667 0.77866667
 0.78533333 0.808      0.804      0.80266667]
[0.68       0.70133333 0.672      0.72133333 0.69066667 0.68533333
 0.68533333 0.70266667 0.716      0.676     ]
[0.832      0.78933333 0.79733333 0.83733333 0.796      0.78533333
 0.80666667 0.80666667 0.80266667 0.80933333]
[0.784      0.76933333 0.756      0.77066667 0.75866667 0.74533333
 0.76533333 0.74933333 0.77466667 0.788     ]


,train accuracy,k-fold-cv mean,k-fold-cv std,Logistic Regressor,DecisionTree Classifier,RandomForest Classifier,AdaBoost Classifier,GradientBoost Classifier,Bagging Classifier
0,NaN,NaN,NaN,0.895371,1.000000,0.804229,1.000000,0.860629,0.988457
1,NaN,NaN,NaN,0.846000,0.694000,0.795200,0.693067,0.806267,0.766133
2,NaN,NaN,NaN,0.009821,0.014536,0.013613,0.015864,0.016001,0.013323
